In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image
from tqdm.notebook import tqdm
import segmentation_models_pytorch as smp
import segmentation_models_pytorch.losses as losses
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
torch.cuda.empty_cache()

In [ ]:
DATA_DIR = '../new_data_lcc'
MODEL_NAME = 'SegformerJaccardLoss'

SAR_DIR = os.path.join(DATA_DIR, 'sar')
MASK_DIR = os.path.join(DATA_DIR, 'ground_truth')
WEIGHTS_DIR = '../weights'
PREDICTIONS_DIR = os.path.join('../predictions_lcc', MODEL_NAME)

os.makedirs(WEIGHTS_DIR, exist_ok=True)
os.makedirs(PREDICTIONS_DIR, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
EPOCHS = 100
PATIENCE = 10
LEARNING_RATE = 1e-4
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
N_CHANNELS = 3

COLOR_MAP = {
    (65, 155, 223): 1,   # 0x419bdf -> Water
    (57, 125, 73): 2,    # 0x397d49 -> Trees
    (122, 135, 198): 4,  # 0x7a87c6 -> Flooded Vegetation
    (228, 150, 53): 5,   # 0xe49635 -> Crops
    (196, 40, 27): 7,    # 0xc4281b -> Built Area
    (165, 155, 143): 8,  # 0xa59b8f -> Bare Ground
    #(168, 235, 255): 9,  # 0xa8ebff -> Snow/Ice
    #(97, 97, 97): 10,    # 0x616161 -> Clouds
    (227, 226, 195): 11, # 0xe3e2c3 -> Rangeland
}

CLASS_LABELS = [1, 2, 4, 5, 7, 8, 11]
LABEL_TO_INDEX = {label: i for i, label in enumerate(CLASS_LABELS)}
INDEX_TO_COLOR = {v: k for k, v in COLOR_MAP.items()}

print(f"Using device: {DEVICE}")
print(f"Number of classes: {len(CLASS_LABELS)}")

In [ ]:
class LandCoverDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_ids, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_ids = image_ids
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)

    def _rgb_to_mask(self, rgb_mask):
        mask = np.zeros((rgb_mask.shape[0], rgb_mask.shape[1]), dtype=np.int64)
        for color, label in COLOR_MAP.items():
            locations = np.where(np.all(rgb_mask == color, axis=-1))
            mask[locations] = LABEL_TO_INDEX[label]
        return torch.from_numpy(mask)

    def __getitem__(self, idx):
        img_name = self.image_ids[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name)

        image = Image.open(img_path).convert("RGB")
        mask_rgb = np.array(Image.open(mask_path).convert("RGB"))

        mask = self._rgb_to_mask(mask_rgb)

        if self.transform:
            image, mask = self.transform(image, mask)

        return image, mask
    
class TrainTransform:
    def __call__(self, image, mask):
        if random.random() > 0.5:
            image = TF.hflip(image)
            mask = TF.hflip(mask)

        if random.random() > 0.5:
            image = TF.vflip(image)
            mask = TF.vflip(mask)

        angle = random.choice([0, 90, 180, 270])
        image = TF.rotate(image, angle)
        mask = TF.rotate(mask.unsqueeze(0), angle, interpolation=transforms.InterpolationMode.NEAREST).squeeze(0)

        image = TF.to_tensor(image)
        image = TF.normalize(image, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        return image, mask

class ValTestTransform:
    def __call__(self, image, mask):
        image = TF.to_tensor(image)
        image = TF.normalize(image, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

        return image, mask

In [ ]:
all_files = sorted([f for f in os.listdir(SAR_DIR) if f.endswith('.png')])
random.seed(42)
random.shuffle(all_files)

n_files = len(all_files)
train_split = int(n_files * 0.7)
val_split = int(n_files * 0.85)

train_ids = all_files[:train_split]
val_ids = all_files[train_split:val_split]
test_ids = all_files[val_split:]

train_dataset = LandCoverDataset(SAR_DIR, MASK_DIR, train_ids, transform=TrainTransform())
val_dataset = LandCoverDataset(SAR_DIR, MASK_DIR, val_ids, transform=ValTestTransform())
test_dataset = LandCoverDataset(SAR_DIR, MASK_DIR, test_ids, transform=ValTestTransform())

num_workers = 4 if torch.cuda.is_available() else 0
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

print(f"Total images: {n_files}")
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

In [ ]:
def train_fn(loader, model, optimizer, loss_fn):
    loop = tqdm(loader, desc="Training")
    model.train()
    total_loss = 0

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.to(device=DEVICE, dtype=torch.long)

        predictions = model(data)
        loss = loss_fn(predictions, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())
        
    return total_loss / len(loader)


def evaluate_model(loader, model, loss_fn):
    num_correct = 0
    num_pixels = 0
    total_loss = 0
    model.eval()

    with torch.no_grad():
        loop = tqdm(loader, desc="Evaluating")
        for data, targets in loop:
            data = data.to(device=DEVICE)
            targets = targets.to(device=DEVICE, dtype=torch.long)
            
            predictions = model(data)
            loss = loss_fn(predictions, targets)
            total_loss += loss.item()

            preds = torch.argmax(predictions, dim=1)
            num_correct += (preds == targets).sum()
            num_pixels += torch.numel(preds)
            
            loop.set_postfix(loss=loss.item(), accuracy=f"{(num_correct/num_pixels)*100:.2f}%")

    accuracy = (num_correct / num_pixels) * 100
    avg_loss = total_loss / len(loader)
    return avg_loss, accuracy

In [ ]:
model = smp.Segformer(encoder_name='efficientnet-b7', classes=len(CLASS_LABELS)).to(DEVICE)
# loss_fn = nn.CrossEntropyLoss()
loss_fn = losses.JaccardLoss(mode='multiclass')

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

best_val_loss = float('inf')
model_save_path = os.path.join(WEIGHTS_DIR, f"{MODEL_NAME}.pth")

In [ ]:
epochs_no_improve = 0

print("Starting training...")

for epoch in range(EPOCHS):
    print(f"\n--- Epoch {epoch+1}/{EPOCHS} ---")
    
    train_loss = train_fn(train_loader, model, optimizer, loss_fn)
    val_loss, val_accuracy = evaluate_model(val_loader, model, loss_fn)
    
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Accuracy={val_accuracy:.2f}%")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), model_save_path)
        print(f"Validation loss improved. Model saved to {model_save_path}")
    else:
        epochs_no_improve += 1
        print(f"No improvement in validation loss for {epochs_no_improve} epoch(s).")
        if epochs_no_improve >= PATIENCE:
            print(f"Early stopping triggered. No improvement in {PATIENCE} epochs.")
            break


print("\n--- Training Finished ---")

In [ ]:
def mask_to_rgb(mask_tensor, class_map):
    mask_np = mask_tensor.cpu().numpy()
    rgb_image = np.zeros((mask_np.shape[0], mask_np.shape[1], 3), dtype=np.uint8)
    
    for class_idx, color in class_map.items():
        original_label = CLASS_LABELS[class_idx]
        rgb_color = INDEX_TO_COLOR[original_label]
        rgb_image[mask_np == class_idx] = rgb_color
        
    return Image.fromarray(rgb_image)

CLASS_NAMES = [
    "Water",
    "Trees",
    "Flooded Vegetation",
    "Crops",
    "Built Area",
    "Bare Ground",
    "Rangeland",
]

epsilon = 1e-6
conf_matrix = np.zeros((len(CLASS_LABELS), len(CLASS_LABELS)), dtype=np.int64)
total_correct = 0
total_pixels = 0

model.load_state_dict(torch.load(model_save_path, weights_only=True))
print(f"Loaded best weights from {model_save_path} for final evaluation.")

print(f"\nSaving test predictions to {PREDICTIONS_DIR} and evaluating metrics...")
model.eval()
with torch.no_grad():
    for i, (x, y) in enumerate(tqdm(test_dataset, desc="Evaluating Model")):
        x = x.unsqueeze(0).to(DEVICE)
        y = y.to(DEVICE)

        preds = torch.argmax(model(x), dim=1).squeeze(0)
        y_true = y.squeeze(0)

        pred_rgb = mask_to_rgb(preds, {i: v for i, v in enumerate(CLASS_LABELS)})
        original_filename = test_ids[i]
        pred_rgb.save(os.path.join(PREDICTIONS_DIR, original_filename))

        preds_np = preds.cpu().numpy().flatten()
        y_np = y_true.cpu().numpy().flatten()

        total_correct += (preds_np == y_np).sum()
        total_pixels += y_np.size

        conf_matrix += confusion_matrix(y_np, preds_np, labels=list(range(len(CLASS_LABELS))))

tp = np.diag(conf_matrix)

fp = np.sum(conf_matrix, axis=0) - tp

fn = np.sum(conf_matrix, axis=1) - tp

iou_per_class = tp / (tp + fp + fn + epsilon)
dice_per_class = (2 * tp) / (2 * tp + fp + fn + epsilon)

pixel_accuracy = total_correct / total_pixels
mean_iou = np.nanmean(iou_per_class)
mean_dice = np.nanmean(dice_per_class)

print("\n--- Test Set Performance ---")
print(f"Test Pixel Accuracy: {pixel_accuracy * 100:.2f}%")
print(f"Mean IoU (from aggregated matrix): {mean_iou:.4f}")
print(f"Mean Dice Coefficient (from aggregated matrix): {mean_dice:.4f}")

print("\n--- Per-Class Metrics (from aggregated matrix) ---")
for i, label in enumerate(CLASS_LABELS):
    print(f"{label}: IoU = {iou_per_class[i]:.4f}, Dice = {dice_per_class[i]:.4f}")

output_dir = '../results'
os.makedirs(output_dir, exist_ok=True)

metrics = {
    'accuracy': pixel_accuracy,
    'iou': mean_iou,
    'dice': mean_dice
}
metrics_path = os.path.join(output_dir, f'{MODEL_NAME}.json')
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=4)
print(f"\nSaved metrics to {metrics_path}")

cm_df = pd.DataFrame(conf_matrix,
                     index = CLASS_NAMES,
                     columns = CLASS_NAMES)

# Plot the confusion matrix using a heatmap
plt.figure(figsize=(10, 8))
ax = sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')

plt.title('Confusion Matrix')
plt.ylabel('True Class')
plt.xlabel('Predicted Class')
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.yticks(rotation=0, fontsize=8)
plt.show()

cm_img_path = os.path.join(output_dir, f'{MODEL_NAME}.png')
fig = ax.get_figure()
fig.savefig(cm_img_path, bbox_inches='tight')
print(f"Saved confusion matrix heatmap to {cm_img_path}")